# Part I. ETL Pipeline for Pre-Processing the Files

## RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [39]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [40]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [41]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [42]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Fields name list

 - 0 : artist
 - 1 : firstName
 - 2 : gender
 - 3 : itemInSession
 - 4 : lastName
 - 5 : length
 - 6 : level
 - 7 : location
 - 8 : sessionId
 - 9 : song
 - 10 : userId

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [43]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [44]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify  
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [45]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

**QUERY 1:** Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

**QUERY 2:** Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
**QUERY 3:** Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## QUERY 1 - Select artist, song and song's length

### CREATE table

In [46]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query1 = """CREATE TABLE IF NOT EXISTS artist_session 
            (sessionId int, itemInSession int, artist text, song text, length float, 
            PRIMARY KEY (sessionId, itemInSession))"""

try:
    session.execute(query1)
except Exception as e:
    print(e)

### INSERT registers

In [47]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## INSERT QUERY
        query = "INSERT INTO artist_session (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)

### SELECT to verify that the data have been inserted into each table

In [48]:
## QUERY1 DEFINITION
query1 = "SELECT artist, song, length FROM artist_session WHERE sessionId=338 AND itemInSession=4"

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


**QUERY 1 DESCRIPTION**

This Query returns artist and song names with the song length for the 'sessionId = 338', and 'itemInSession = 4'.

This query needs a table (artist_session) with the columns artist, song, and length and the sessionId and itemInSession columns as the keys for the filtering. sessionId and itemInSession are needed as the Primary key to guaranty the uniqueness due sessionId by its one is not unique. sessionId column is defined as the partition key because data is distributed by session.

## QUERY 2 - Select artist, song and user (first and last name)

### CREATE table

In [49]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query2 = """CREATE TABLE IF NOT EXISTS artist_user_session 
            (userid int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, 
            PRIMARY KEY ((userid, sessionId), itemInSession)) 
            WITH CLUSTERING ORDER BY (itemInSession ASC);"""

try:
    session.execute(query2)
except Exception as e:
    print(e)

### INSERT registers

In [50]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #Insert sql script
        query2 = "INSERT INTO artist_user_session (userid, sessionId, itemInSession, artist, song, firstName, lastName)"
        query2 = query2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

        try:
            session.execute(query2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)            

### SELECT to verify that the data have been inserted into each table

In [51]:
## Add in the SELECT statement to verify the data was entered into the table

## QUERY2 DEFINITION
query2 = "SELECT artist, song, firstName, lastName FROM artist_user_session WHERE userid = 10 AND sessionid = 182"

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


**QUERY 2 DESCRIPTION**

This Query returns artist, song and user names (First and last name) filtered by userid and sessionid. 

This query needs a table (artist_user_session) with the columns artist, song, firstName and lastName and the columns userId and sessionId as keys for the filtering. userid and sessionid are needed as the Primary key to guaranty the uniqueness and userIf used as the partition key.

## QUERY 3 - Select user name (first and last) in my music app history who listened to the song

### CREATE table

In [52]:
## Query 3: Give me every user name (first and last) in my music app history who listened 
## to the song 'All Hands Against His Own'

query3 = """CREATE TABLE IF NOT EXISTS user_by_song 
            (song text, userId int, firstName text, lastName text,  
            PRIMARY KEY (song, userid))"""

try:
    session.execute(query3)
except Exception as e:
    print(e)

### INSERT registers

In [53]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Insert QUERY
        query3 = "INSERT INTO user_by_song (song, userId, firstName, lastName)"
        query3 = query3 + " VALUES (%s, %s, %s, %s)"
        
        try:
            session.execute(query3, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)

### SELECT to verify that the data have been inserted into each table

In [54]:
## QUERY3 DEFINITION
query3 = "SELECT userId, firstName, lastName FROM user_by_song WHERE song='All Hands Against His Own'"

try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.userid, row.firstname, row.lastname)

29 Jacqueline Lynch
80 Tegan Levine
95 Sara Johnson


**QUERY 3 DESCRIPTION**

This Query returns userid, firtName and lastName filtered by song and song.

This query needs a table (user_by_song) with the columns firstName and lastName and userId for clustering and song for partition key. The key userId is required to guaranty uniqueness in the results.

### Drop the tables before closing out the sessions

In [55]:
## Drop table artist_session
try:
    rows = session.execute("DROP TABLE artist_session")
except Exception as e:
    print(e)

In [56]:
## Drop table artist_user_session
try:
    rows = session.execute("DROP TABLE artist_user_session")
except Exception as e:
    print(e)

In [ ]:
## Drop table artist_session
try:
    rows = session.execute("DROP TABLE user_by_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()